In [ ]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkConf

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Algorithms
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

# Others
from collections import OrderedDict

# Graphs libs
import matplotlib.pyplot as plt

# Supress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Spark context simple configuration
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

spark.version

In [ ]:
%load_ext autoreload
%autoreload 2

from Helpers.technical_indicators import calc_ti
from Helpers.generated_features import features_from_OHLC
from Helpers.CustomTS import TrainValidationSplitSorted
from Helpers.best_model_params import *

from ProcessingData.processing import initial_processing, calc_profit, transform_date, train_test_split, complete_processing
from Stats.measures import calc_metrics

In [ ]:
ManualSplit = True
SORT = True
CHUNKS = 10

CV = False

DEBUG = False
RANDOM_SEED = 1

In [ ]:
# path_to_csv = "s3://stocksets100/Orlen.csv"
path = "./Datasets/KGHA.csv"
df = complete_processing(spark, path)

train, test = train_test_split(spark, df, CHUNKS, SORT, ManualSplit, RANDOM_SEED)

In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
featuresCols = df.columns
featuresCols.remove('Profit')
featuresCols.remove('id')

print(featuresCols)

# Vector Assembler
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
# Used for assembling features into a vector.
# We will pass all the columns that we are going to use for the prediction to the VectorAssembler and
# it will create a new vector column.
vectorAssembler_rt = VectorAssembler(
    inputCols=featuresCols, outputCol="features")

In [ ]:
# Patrameters grid testing
# rt = DecisionTreeClassifier(
#     labelCol='Profit', featuresCol="features", minInfoGain=0.01,  maxBins=200)

rt = RandomForestClassifier(
    labelCol='Profit', featuresCol="features", numTrees=10, maxBins=300)

max_Depth_Range = [3]
min_InstancesPerNode = list(range(5, 15))

paramGrid = ParamGridBuilder() \
    .addGrid(rt.maxDepth, max_Depth_Range) \
    .addGrid(rt.maxMemoryInMB, [1000] ).build()

# We define an evaluation metric. This tells Validator how well we are doing by comparing the true
# labels with predictions.

evaluator = MulticlassClassificationEvaluator(
    labelCol=rt.getLabelCol(),
    metricName='accuracy',
    predictionCol=rt.getPredictionCol())

In [ ]:
# Declare the CrossValidator, which runs model tuning for us.
if CV:
    val = CrossValidator(
        estimator=rt,
        evaluator=multi,
        estimatorParamMaps=paramGrid,
        numFolds=2)
else:
    val = TrainValidationSplitSorted(
        chunks = CHUNKS,
        spark = spark,
        estimator=rt,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator)

In [ ]:
# Creating Final pipeline object
pipeline_rt = Pipeline(stages=[vectorAssembler_rt, val])

# FITTING!
pipelineModel_rt = pipeline_rt.fit(train)

# Getting the Best Model
best_classifier = pipelineModel_rt.stages[-1].bestModel

In [ ]:
# Feature importance
# print('Features importances' + str(best_classifier.featureImportances))
final_features = best_classifier.featureImportances

feature_dict = {}
for feature, importance in zip(featuresCols, final_features):
    feature_dict[feature] = importance

feature_dict = OrderedDict(sorted(feature_dict.items(), key=lambda t: t[1], reverse=True)) 

i = 1
for feature, importance in feature_dict.items():
    print("{} ; {} ; {}".format(i, feature, round(importance, 3)))
    i+=1

In [ ]:
# Making Predictions!
predictions = pipelineModel_rt.transform(test)

In [ ]:
#evaluate results
calc_metrics(predictions)

In [ ]:
if DEBUG != True:
    df_to_plot_rt = predictions.select('prediction', 'Profit')
    df_to_plot_rt = df_to_plot_rt.toPandas()
    plt.figure(figsize=(24, 3))
    plt.plot(df_to_plot_rt)
    plt.legend(df_to_plot_rt.columns)
    plt.show()

In [ ]:
i = 0
# path_to_csv = "s3://stocksets100/Orlen.csv"
path = "./Datasets/KGHA.csv"
df = complete_processing(spark, path)
ManualSplit = False
for i in range(30):
    train, test = train_test_split(spark, df, CHUNKS, SORT, ManualSplit, RANDOM_SEED + i)
    predictions = pipelineModel_rt.transform(test)
    calc_metrics(predictions)
    i+=1
    print("#########################################################################")

We have 909 training examples and 398 test examples.
Summary Stats
DenseMatrix([[ 163.,   47.],
             [ 110.,   78.]])
Accuracy = 0.6055
Recall = 0.6055
F1 Score = 0.6055


#########################################################################
We have 904 training examples and 403 test examples.
Summary Stats
DenseMatrix([[ 174.,   43.],
             [ 109.,   77.]])
Accuracy = 0.6228
Recall = 0.6228
F1 Score = 0.6228


#########################################################################
We have 905 training examples and 402 test examples.
Summary Stats
DenseMatrix([[ 170.,   48.],
             [ 100.,   84.]])
Accuracy = 0.6318
Recall = 0.6318
F1 Score = 0.6318


#########################################################################
We have 907 training examples and 400 test examples.
Summary Stats
DenseMatrix([[ 161.,   41.],
             [ 106.,   92.]])
Accuracy = 0.6325
Recall = 0.6325
F1 Score = 0.6325


##########################################################